In [1]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses

In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.words))
        print(self.words)
        print(self.speakers)

ea=EasyASR()
print(ea.leWords)

Imports doned
fNames= 108
16000 wave= (3871,) mfcc= (8, 20) file= /home/francisco/voz/paco_no_006.wav
16000 wave= (4963,) mfcc= (10, 20) file= /home/francisco/voz/paco_si_008.wav
16000 wave= (9285,) mfcc= (19, 20) file= /home/francisco/voz/paco_dos_002.wav
16000 wave= (4434,) mfcc= (9, 20) file= /home/francisco/voz/paco_uno_004.wav
16000 wave= (2428,) mfcc= (5, 20) file= /home/francisco/voz/paco_no_007.wav
16000 wave= (4882,) mfcc= (10, 20) file= /home/francisco/voz/paco_para_003.wav
16000 wave= (6972,) mfcc= (14, 20) file= /home/francisco/voz/paco_cuatro_003.wav
16000 wave= (6524,) mfcc= (13, 20) file= /home/francisco/voz/paco_nueve_001.wav
16000 wave= (4630,) mfcc= (10, 20) file= /home/francisco/voz/carlos_si_002.wav
16000 wave= (14450,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_001.wav
16000 wave= (14816,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_003.wav
16000 wave= (6847,) mfcc= (14, 20) file= /home/francisco/voz/irene_hola_003.wav
16000 wave= (9176

In [3]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

17 4
(108, 30, 20) (108, 17) (108, 4)


In [65]:
# Network building
def build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=128):
    input = Input(shape=(None,input_dim))

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim)(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim)(x)
    sd= Dense(hiden_dim)(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = Dense(outputWords_dim   , activation="softmax", name="word")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)

    model = Model(input, [prediction_words,prediction_speakers])
    return model

In [66]:
model=build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, None, 128)    7808        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_13 (Dropout)            (None, None, 128)    0           conv1d_10[0][0]                  
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, None, 128)    49280       dropout_13[0][0]                 
____________________________________________________________________________________________

In [33]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=108,epochs=5000,validation_split=0.2)

Train on 86 samples, validate on 22 samples
Epoch 1/5000
86/86 [==============================] - 0s 193us/sample - loss: 0.5390 - word_loss: 0.5382 - speaker_loss: 8.3534e-04 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 3.4797 - val_word_loss: 3.2288 - val_speaker_loss: 0.2508 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 2/5000
86/86 [==============================] - 0s 167us/sample - loss: 0.0154 - word_loss: 0.0151 - speaker_loss: 3.1855e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 3.3182 - val_word_loss: 3.0803 - val_speaker_loss: 0.2379 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 3/5000
86/86 [==============================] - 0s 162us/sample - loss: 0.0054 - word_loss: 0.0046 - speaker_loss: 7.9770e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 3.1392 - val_word_loss: 2.9098 - val_speaker_loss: 0.2294 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4/5000
86/86 [

Epoch 28/5000
86/86 [==============================] - 0s 215us/sample - loss: 0.0028 - word_loss: 0.0023 - speaker_loss: 4.9328e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.4794 - val_word_loss: 2.2040 - val_speaker_loss: 0.2754 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9545
Epoch 29/5000
86/86 [==============================] - 0s 200us/sample - loss: 0.1605 - word_loss: 0.1599 - speaker_loss: 5.2852e-04 - word_accuracy: 0.9535 - speaker_accuracy: 1.0000 - val_loss: 2.4946 - val_word_loss: 2.2159 - val_speaker_loss: 0.2788 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9545
Epoch 30/5000
86/86 [==============================] - 0s 224us/sample - loss: 0.0069 - word_loss: 0.0064 - speaker_loss: 5.0830e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.5252 - val_word_loss: 2.2421 - val_speaker_loss: 0.2831 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9545
Epoch 31/5000
86/86 [==============================] - 0s 223

Epoch 55/5000
86/86 [==============================] - 0s 206us/sample - loss: 0.1344 - word_loss: 0.1197 - speaker_loss: 0.0147 - word_accuracy: 0.9767 - speaker_accuracy: 0.9884 - val_loss: 3.5229 - val_word_loss: 3.0858 - val_speaker_loss: 0.4371 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.8636
Epoch 56/5000
86/86 [==============================] - 0s 198us/sample - loss: 0.0486 - word_loss: 0.0481 - speaker_loss: 4.4969e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 3.5351 - val_word_loss: 3.1049 - val_speaker_loss: 0.4303 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.8636
Epoch 57/5000
86/86 [==============================] - 0s 170us/sample - loss: 0.0267 - word_loss: 0.0201 - speaker_loss: 0.0066 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 3.5435 - val_word_loss: 3.1192 - val_speaker_loss: 0.4242 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.8636
Epoch 58/5000
86/86 [==============================] - 0s 171us/sampl

Epoch 82/5000
86/86 [==============================] - 0s 185us/sample - loss: 0.0034 - word_loss: 0.0030 - speaker_loss: 4.0032e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 3.5518 - val_word_loss: 3.2473 - val_speaker_loss: 0.3045 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 83/5000
86/86 [==============================] - 0s 166us/sample - loss: 0.0651 - word_loss: 0.0645 - speaker_loss: 6.5299e-04 - word_accuracy: 0.9767 - speaker_accuracy: 1.0000 - val_loss: 3.5836 - val_word_loss: 3.2837 - val_speaker_loss: 0.2999 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 84/5000
86/86 [==============================] - 0s 161us/sample - loss: 0.0659 - word_loss: 0.0628 - speaker_loss: 0.0031 - word_accuracy: 0.9767 - speaker_accuracy: 1.0000 - val_loss: 3.6027 - val_word_loss: 3.3074 - val_speaker_loss: 0.2953 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 85/5000
86/86 [==============================] - 0s 168us/s

Epoch 109/5000
86/86 [==============================] - 0s 160us/sample - loss: 0.0077 - word_loss: 0.0054 - speaker_loss: 0.0023 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 4.1527 - val_word_loss: 3.8128 - val_speaker_loss: 0.3399 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 110/5000
86/86 [==============================] - 0s 160us/sample - loss: 0.2066 - word_loss: 0.0730 - speaker_loss: 0.1336 - word_accuracy: 0.9884 - speaker_accuracy: 0.9767 - val_loss: 4.1810 - val_word_loss: 3.8510 - val_speaker_loss: 0.3300 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 111/5000
86/86 [==============================] - 0s 173us/sample - loss: 0.1243 - word_loss: 0.1077 - speaker_loss: 0.0166 - word_accuracy: 0.9884 - speaker_accuracy: 0.9884 - val_loss: 4.1839 - val_word_loss: 3.8655 - val_speaker_loss: 0.3184 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 112/5000
86/86 [==============================] - 0s 169us/sampl

...

86/86 [==============================] - 0s 183us/sample - loss: 0.1049 - word_loss: 0.0180 - speaker_loss: 0.0869 - word_accuracy: 0.9884 - speaker_accuracy: 0.9884 - val_loss: 2.9965 - val_word_loss: 2.5967 - val_speaker_loss: 0.3998 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4886/5000
86/86 [==============================] - 0s 185us/sample - loss: 0.0016 - word_loss: 0.0010 - speaker_loss: 6.1063e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.9749 - val_word_loss: 2.5898 - val_speaker_loss: 0.3851 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4887/5000
86/86 [==============================] - 0s 159us/sample - loss: 0.0037 - word_loss: 0.0036 - speaker_loss: 3.8207e-05 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.9579 - val_word_loss: 2.5804 - val_speaker_loss: 0.3776 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4888/5000
86/86 [==============================] - 0s 172us/sample - 

Epoch 4912/5000
86/86 [==============================] - 0s 179us/sample - loss: 0.0012 - word_loss: 9.7223e-04 - speaker_loss: 2.4102e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 3.0074 - val_word_loss: 2.4672 - val_speaker_loss: 0.5402 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4913/5000
86/86 [==============================] - 0s 195us/sample - loss: 0.0016 - word_loss: 6.8513e-04 - speaker_loss: 9.2677e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 3.0121 - val_word_loss: 2.4655 - val_speaker_loss: 0.5466 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4914/5000
86/86 [==============================] - 0s 176us/sample - loss: 0.0318 - word_loss: 0.0314 - speaker_loss: 3.7111e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 3.0111 - val_word_loss: 2.4610 - val_speaker_loss: 0.5501 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4915/5000
86/86 [========================

86/86 [==============================] - 0s 182us/sample - loss: 0.0165 - word_loss: 0.0154 - speaker_loss: 0.0012 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.6238 - val_word_loss: 2.0979 - val_speaker_loss: 0.5259 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4939/5000
86/86 [==============================] - 0s 172us/sample - loss: 0.0323 - word_loss: 0.0319 - speaker_loss: 3.8942e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.6300 - val_word_loss: 2.1064 - val_speaker_loss: 0.5235 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4940/5000
86/86 [==============================] - 0s 170us/sample - loss: 0.1491 - word_loss: 0.1486 - speaker_loss: 5.2653e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.6215 - val_word_loss: 2.1148 - val_speaker_loss: 0.5067 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4941/5000
86/86 [==============================] - 0s 182us/sample - 

Epoch 4965/5000
86/86 [==============================] - 0s 166us/sample - loss: 0.0037 - word_loss: 0.0033 - speaker_loss: 3.8203e-04 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.0910 - val_word_loss: 1.8265 - val_speaker_loss: 0.2645 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4966/5000
86/86 [==============================] - 0s 161us/sample - loss: 0.1675 - word_loss: 0.1666 - speaker_loss: 9.3464e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.0698 - val_word_loss: 1.8095 - val_speaker_loss: 0.2604 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4967/5000
86/86 [==============================] - 0s 167us/sample - loss: 0.0270 - word_loss: 0.0257 - speaker_loss: 0.0012 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.0643 - val_word_loss: 1.8062 - val_speaker_loss: 0.2581 - val_word_accuracy: 0.8182 - val_speaker_accuracy: 0.9091
Epoch 4968/5000
86/86 [==============================] - 0s

Epoch 4992/5000
86/86 [==============================] - 0s 155us/sample - loss: 0.0461 - word_loss: 0.0456 - speaker_loss: 5.2155e-04 - word_accuracy: 0.9884 - speaker_accuracy: 1.0000 - val_loss: 2.5847 - val_word_loss: 2.0867 - val_speaker_loss: 0.4980 - val_word_accuracy: 0.7727 - val_speaker_accuracy: 0.9091
Epoch 4993/5000
86/86 [==============================] - 0s 154us/sample - loss: 0.0017 - word_loss: 0.0016 - speaker_loss: 7.7569e-05 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 2.5705 - val_word_loss: 2.0794 - val_speaker_loss: 0.4911 - val_word_accuracy: 0.7727 - val_speaker_accuracy: 0.9091
Epoch 4994/5000
86/86 [==============================] - 0s 154us/sample - loss: 0.1645 - word_loss: 0.1640 - speaker_loss: 5.0644e-04 - word_accuracy: 0.9767 - speaker_accuracy: 1.0000 - val_loss: 2.5706 - val_word_loss: 2.0845 - val_speaker_loss: 0.4861 - val_word_accuracy: 0.7727 - val_speaker_accuracy: 0.9091
Epoch 4995/5000
86/86 [==============================] 

In [34]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 30, 20)
[[1.6032067e-06 1.5474246e-06 8.5649139e-05 9.5820587e-06 1.0769221e-06
  3.2579137e-05 2.5468027e-05 8.4635349e-06 2.6492586e-05 3.6695317e-06
  9.9977034e-01 7.1688391e-06 7.0926984e-07 2.0389862e-06 1.0135352e-05
  1.2663489e-05 8.5942378e-07]] [[4.1266088e-04 2.1086236e-04 1.0212685e-04 9.9927443e-01]]
10 = 0.99977034
3 = 0.99927443
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']


In [59]:
!sox -r 16000 -t alsa default recording.wav silence 1 0.1 1% 1 1.5 1%
!aplay "recording.wav"


Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 16000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:03.33 [00:00:00.00] Out:15.4k [      |      ]        Clip:0    
Done.
Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [60]:
!aplay "recording.wav"

Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [61]:
sample=wav2mfcc("recording.wav")
sample=np.expand_dims(np.array(sample),axis=0)
sample.shape

16000 wave= (15379,) mfcc= (31, 20) file= recording.wav


(1, 30, 20)

In [62]:
predWord,predSpeaker=model.predict([sample])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

[[1.5683449e-28 5.8647111e-19 2.1280541e-28 9.5844884e-32 6.5951492e-28
  3.8340293e-18 6.5511953e-35 1.4257799e-34 2.2104685e-24 1.2992054e-23
  1.6596311e-38 1.3464838e-23 1.0000000e+00 1.0043820e-21 1.1489732e-18
  1.0021130e-20 4.3862555e-18]] [[2.1934634e-20 4.8941130e-20 4.3469758e-19 1.0000000e+00]]
12 = 1.0
3 = 1.0
paco  ha dicho ' seis '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
